In [1]:
import torch
import pickle
import random
import numpy as np
import main_data_representation as mdr

In [2]:
# seed 고정
random_seed = 42

torch.manual_seed(random_seed)
torch.cuda.manual_seed(random_seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(random_seed)
random.seed(random_seed)

In [3]:
# Case 1. model = ts2vec
config1 = {
    "model": 'ts2vec',
    "training": True,  # 학습 여부, 저장된 학습 완료 모델 존재시 False로 설정
    "best_model_path": './ckpt/ts2vec.pt',  # 학습 완료 모델 저장 경로
    "parameter": {
        "input_dim": 9,  # 데이터의 변수 개수, int
        "repr_dim": 64,  # data representation 차원, int(default: 64, 범위: 1 이상, 2의 지수로 설정 권장)
        "hidden_dim": 64,  # encoder의 hidden dimension, int(default: 64, 범위: 1 이상, default 값 사용 권장)
        "num_epochs": 30,  # 학습 epoch 횟수, int(default: 30, 범위: 1 이상)
        "batch_size": 512,  # batch 크기, int(default: 512, 범위: 1 이상, 컴퓨터 사양에 적합하게 설정)
        "lr": 0.001,  # learning rate, float(default: 0.001, 범위: 0.1 이하)
        "device": "cuda",  # 학습 환경, ["cuda", "cpu"] 중 선택
    }
}

# Case 2. model = ts_tcc
config2 = {
    "model": 'ts_tcc',
    "training": True,  # 학습 여부, 저장된 학습 완료 모델 존재시 False로 설정
    "best_model_path": './ckpt/ts_tcc.pt',  # 학습 완료 모델 저장 경로
    "parameter": {
        "input_dim": 9,  # 데이터의 변수 개수, int
        "repr_dim": 64,  # data representation 차원, int(default: 64, 범위: 1 이상, 2의 지수로 설정 권장)
        "hidden_dim": 100,  # temporal / contextual contrasting 모듈의 hidden dimension, int(default: 100, 범위: 1 이상, default 값 사용 권장)
        "timesteps": 6,  # temporal contrasting 모듈에서 미래 예측할 시점의 길이, int(default: 6, 범위: 1 이상)
        "num_epochs": 30,  # 학습 epoch 횟수, int(default: 30, 범위: 1 이상)
        "batch_size": 512,  # batch 크기, int(default: 512, 범위: 1 이상, 컴퓨터 사양에 적합하게 설정)
        "lr": 0.0001,  # learning rate, float(default: 0.0001, 범위: 0.1 이하)
        "device": "cuda",  # 학습 환경, ["cuda", "cpu"] 중 선택
        "jitter_scale_ratio": 1.1,  # time series data augementation 중 weak augementation의 강도, float(default: 1.1, default 값 사용 권장)
        "jitter_ratio": 0.8,  # time series data augementation 중 strong augementation의 강도, float(default: 0.8, default 값 사용 권장)
        "max_seg": 8  # strong augementation에서 permutation 진행시 데이터의 최대 분할 개수, int(default: 8, default 값 사용 권장)
    }
}

# Case 3. model = rae_mepc
config3 = {
    "model": 'rae_mepc',
    "training": True,  # 학습 여부, 저장된 학습 완료 모델 존재시 False로 설정
    "best_model_path": './ckpt/rae_mepc.pt',  # 학습 완료 모델 저장 경로
    "parameter": {
        "window_size": 32,  # 모델의 input sequence 길이, int(default: 32, 범위: 0 이상 & 원래 데이터의 sequence 길이 이하)
        "input_dim": 9,  # 데이터의 변수 개수, int
        "repr_dim": 64,  # data representation 차원, int(default: 64, 범위: 1 이상, 2의 지수로 설정 권장)
        "enc_nlayers": 3,  # multi-resolution encoder를 구성하는 sub-encoder의 개수, int(default: 3, 범위: 1 이상, default 값 사용 권장)
        "dec_nlayers": 3,  # multi-resolution decoder를 구성하는 sub-decoder의 개수, int(default: 3, 범위: 1 이상, default 값 사용 권장)
        "tau": 4,  # multi-resolution encoder 및 decoder의 resolution를 조절하는 값, int(default: 4, 범위: 2 이상, default 값 사용 권장)
        "num_epochs": 30,  # 학습 epoch 횟수, int(default: 30, 범위: 1 이상)
        "batch_size": 512,  # batch 크기, int(default: 512, 범위: 1 이상, 컴퓨터 사양에 적합하게 설정)
        "lr": 0.0001,  # learning rate, float(default: 0.0001, 범위: 0.1 이하)
        "device": "cuda"  # 학습 환경, ["cuda", "cpu"] 중 선택
    }
}

# Case 4. model = stoc
config4 = {
    "model": 'stoc',
    "training": True,  # 학습 여부, 저장된 학습 완료 모델 존재시 False로 설정
    "best_model_path": './ckpt/stoc.pt',  # 학습 완료 모델 저장 경로
    "parameter": { 
        "window_size": 32,  # 모델의 input sequence 길이, int(default: 32, 범위: 0 이상 & 원래 데이터의 sequence 길이 이하)
        "input_dim": 9,  # 데이터의 변수 개수, int
        "repr_dim": 64,  # data representation 차원, int(default: 64, 범위: 1 이상, 2의 지수로 설정 권장)
        "hidden_dim": 256,  # encoder의 hidden dimension, int(default: 356, 범위: 1 이상, default 값 사용 권장)
        "forecast_step": 1,  # 미래 시계열 데이터에 대하여 예측할 시점의 길이, int(default: 6, 범위: 1 이상)
        "num_epochs": 30,  # 학습 epoch 횟수, int(default: 30, 범위: 1 이상)
        "batch_size": 512,  # batch 크기, int(default: 512, 범위: 1 이상, 컴퓨터 사양에 적합하게 설정)
        "lr": 0.001,  # learning rate, float(default: 0.001, 범위: 0.1 이하)
        "device": "cuda",  # 학습 환경, ["cuda", "cpu"] 중 선택, 
        "patience": 10,  # 예측 모델 학습 시, 사전 설정한 epoch 동안 loss가 감소하지 않으면 학습 조기 중단, int(default: 10, 범위: 1 이상 num_epochs 미만)
    }
}

In [4]:
# Dataset
dataset_dir = {
    "train": './data/X_train.pkl',
    "test": './data/X_test.pkl'
}

# train/test 데이터 불러오기 (pickle 형태)
# shape=(# observations, # features, # time steps)
train_data = pickle.load(open(dataset_dir["train"], 'rb'))  # shape=(7352, 9, 128)
test_data = pickle.load(open(dataset_dir["test"], 'rb'))  # shape=(2947, 9, 128)

In [5]:
# Case 1. model = ts2vec
config = config1
data_repr = mdr.Encode(config, train_data, test_data)
model = data_repr.build_model()  # 모델 구축

if config["training"]:
    best_model = data_repr.train_model(model)  # 모델 학습
    data_repr.save_model(best_model, best_model_path=config["best_model_path"])  # 모델 저장

train_repr, test_repr = data_repr.encode_data(model, best_model_path=config["best_model_path"])  # representation 도출

Start training model

Epoch #1: train loss=4.936868476867676
Epoch #1: validation loss=4.943576335906982

Epoch #2: train loss=4.627585792541504
Epoch #2: validation loss=4.884347915649414

Epoch #3: train loss=4.79036808013916
Epoch #3: validation loss=4.623116493225098

Epoch #4: train loss=4.55261926651001
Epoch #4: validation loss=4.731514930725098

Epoch #5: train loss=4.5294112205505375
Epoch #5: validation loss=4.675436973571777

Epoch #6: train loss=4.473555088043213
Epoch #6: validation loss=4.2629075050354

Epoch #7: train loss=4.341904354095459
Epoch #7: validation loss=4.463427543640137

Epoch #8: train loss=4.481424331665039
Epoch #8: validation loss=4.20048189163208

Epoch #9: train loss=4.2812458038330075
Epoch #9: validation loss=4.146800994873047

Epoch #10: train loss=4.198643016815185
Epoch #10: validation loss=4.0082621574401855

Epoch #11: train loss=4.109360694885254
Epoch #11: validation loss=4.026050567626953

Epoch #12: train loss=4.023466968536377
Epoch #12: v

In [6]:
print(train_repr.shape, test_repr.shape)

(7352, 64) (2947, 64)


In [7]:
train_repr.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V55,V56,V57,V58,V59,V60,V61,V62,V63,V64
0,-0.033631,0.036715,0.237707,-0.010023,0.490949,1.374596,0.435587,1.136328,0.074970,0.015736,...,0.196531,0.033214,0.691922,-0.014442,0.643810,-0.098838,0.024468,0.000844,-0.024828,-0.003097
1,-0.007252,0.016638,0.240048,-0.012824,0.520046,1.381432,0.450673,1.141469,0.087617,0.013349,...,0.185382,0.017689,0.651348,-0.068286,0.674752,-0.049794,0.073779,-0.019720,-0.023705,-0.000657
2,0.001482,-0.011001,0.213614,-0.007623,0.483143,1.387769,0.392625,1.125404,0.175980,0.050220,...,0.207115,0.011044,0.655682,-0.052771,0.665929,-0.111551,0.075382,0.033315,-0.039400,0.034564
3,-0.035661,-0.008861,0.210229,-0.016049,0.403497,1.410193,0.384614,1.139218,0.157020,0.032520,...,0.201145,-0.025719,0.614282,-0.061299,0.724790,-0.141869,0.085174,0.030246,-0.036670,-0.034799
4,-0.018921,0.028302,0.227644,-0.004378,0.450709,1.409542,0.405561,1.148567,0.159500,0.045878,...,0.212822,-0.015031,0.602995,-0.061823,0.733468,-0.071200,0.087868,0.016841,-0.021300,-0.001932


In [8]:
test_repr.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V55,V56,V57,V58,V59,V60,V61,V62,V63,V64
0,0.406812,0.339256,-0.015329,0.297275,1.282596,1.391934,0.518683,1.089709,-0.049185,0.060886,...,0.128975,0.372564,0.665117,0.226362,0.454335,0.246093,-0.157500,0.084735,0.375224,0.319567
1,0.373252,0.135933,-0.082387,0.335827,1.135910,1.500563,0.396863,1.085935,0.167597,0.115023,...,0.093011,0.251897,0.592738,0.072304,0.548451,0.071378,-0.159463,0.147932,0.246739,0.162483
2,0.350981,-0.038944,-0.107906,0.359263,0.848999,1.496482,0.412876,1.090490,0.203876,0.092680,...,0.058355,0.079587,0.519686,0.001276,0.682945,-0.110836,-0.091623,0.167717,0.195251,-0.022971
3,0.368999,-0.075375,-0.126406,0.401271,0.844119,1.511416,0.346760,1.093723,0.339085,0.083968,...,0.063081,0.101366,0.534796,0.031849,0.660477,-0.153458,-0.127524,0.231389,0.196502,0.031155
4,0.350008,-0.052107,-0.125244,0.417671,0.714224,1.526109,0.359167,1.110752,0.296702,0.079081,...,0.069656,0.087774,0.481167,0.067382,0.734597,-0.208285,-0.087492,0.246985,0.209821,-0.057286


In [10]:
# Case 2. model = ts_tcc
config = config2
data_repr = mdr.Encode(config, train_data, test_data)
model = data_repr.build_model()  # 모델 구축

if config["training"]:
    best_model = data_repr.train_model(model)  # 모델 학습
    data_repr.save_model(best_model, best_model_path=config["best_model_path"])  # 모델 저장

train_repr, test_repr = data_repr.encode_data(model, best_model_path=config["best_model_path"])  # representation 도출

Start training model

Epoch #1: train loss=17.66071319580078
Epoch #2: validation loss=17.68142318725586

Epoch #2: train loss=17.633176803588867
Epoch #3: validation loss=18.060693740844727

Epoch #3: train loss=17.58555793762207
Epoch #4: validation loss=17.806976318359375

Epoch #4: train loss=17.535953521728516
Epoch #5: validation loss=17.560094833374023

Epoch #5: train loss=17.484237670898438
Epoch #6: validation loss=17.826210021972656

Epoch #6: train loss=17.414249420166016
Epoch #7: validation loss=17.420270919799805

Epoch #7: train loss=17.33850860595703
Epoch #8: validation loss=17.34860610961914

Epoch #8: train loss=17.2409725189209
Epoch #9: validation loss=17.15634536743164

Epoch #9: train loss=17.18472671508789
Epoch #10: validation loss=17.230012893676758

Epoch #10: train loss=17.0809268951416
Epoch #11: validation loss=17.014469146728516

Epoch #11: train loss=16.98137664794922
Epoch #12: validation loss=16.992862701416016

Epoch #12: train loss=16.75034141540527

In [11]:
print(train_repr.shape, test_repr.shape)

(7352, 64) (2947, 64)


In [12]:
train_repr.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V55,V56,V57,V58,V59,V60,V61,V62,V63,V64
0,1.415004,1.247389,0.0,1.312446,1.165330,1.543982,0.0,0.0,1.319170,1.327649,...,0.0,0.852361,1.548516,0.0,2.025096,0.184164,1.279742,0.438212,0.0,1.433192
1,1.408585,1.221739,0.0,1.291749,1.155547,1.512117,0.0,0.0,1.333663,1.335803,...,0.0,0.842610,1.529925,0.0,2.009972,0.181050,1.278370,0.388566,0.0,1.435636
2,1.408841,1.210168,0.0,1.291822,1.138199,1.512824,0.0,0.0,1.304309,1.330241,...,0.0,0.850899,1.531653,0.0,2.010880,0.162559,1.278116,0.384694,0.0,1.431300
3,1.410115,1.190142,0.0,1.291264,1.143094,1.502158,0.0,0.0,1.312961,1.360146,...,0.0,0.851072,1.532606,0.0,1.991898,0.166120,1.278798,0.371485,0.0,1.444807
4,1.407546,1.192116,0.0,1.280644,1.142739,1.496653,0.0,0.0,1.327218,1.347511,...,0.0,0.848754,1.534866,0.0,1.993783,0.186403,1.279452,0.369416,0.0,1.434836


In [13]:
test_repr.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V55,V56,V57,V58,V59,V60,V61,V62,V63,V64
0,1.411522,1.319475,0.0,1.135064,1.163418,1.659809,0.0,0.0,1.334034,1.314916,...,0.0,0.823418,1.663288,0.0,2.030751,0.186801,1.265284,0.607910,0.0,1.483165
1,1.422119,1.260183,0.0,1.280201,1.081639,1.498350,0.0,0.0,1.300243,1.384459,...,0.0,0.839122,1.501606,0.0,1.962393,0.125497,1.250816,0.395119,0.0,1.466509
2,1.422595,1.165883,0.0,1.201591,1.091853,1.469469,0.0,0.0,1.284778,1.396889,...,0.0,0.833513,1.522691,0.0,1.925355,0.144013,1.242125,0.309935,0.0,1.459934
3,1.422809,1.175616,0.0,1.227591,1.084457,1.476916,0.0,0.0,1.292325,1.375789,...,0.0,0.829558,1.529028,0.0,1.948569,0.120967,1.240961,0.334961,0.0,1.466668
4,1.426434,1.160866,0.0,1.207253,1.103280,1.468346,0.0,0.0,1.298864,1.397364,...,0.0,0.841721,1.525189,0.0,1.923682,0.150940,1.248361,0.319965,0.0,1.476749


In [15]:
# Case 3. model = rae_mepc
config = config3
data_repr = mdr.Encode(config, train_data, test_data)
model = data_repr.build_model()  # 모델 구축

if config["training"]:
    best_model = data_repr.train_model(model)  # 모델 학습
    data_repr.save_model(best_model, best_model_path=config["best_model_path"])  # 모델 저장

train_repr, test_repr = data_repr.encode_data(model, best_model_path=config["best_model_path"])  # representation 도출

Start training model

Epoch #1: train loss=0.3251935541629791
Epoch #1: validation loss=0.15816469490528107

Epoch #2: train loss=0.305441677570343
Epoch #2: validation loss=0.1445966213941574

Epoch #3: train loss=0.28639599680900574
Epoch #3: validation loss=0.13168644905090332

Epoch #4: train loss=0.2673955261707306
Epoch #4: validation loss=0.11976871639490128

Epoch #5: train loss=0.248679518699646
Epoch #5: validation loss=0.10822036862373352

Epoch #6: train loss=0.2331651747226715
Epoch #6: validation loss=0.10381440818309784

Epoch #7: train loss=0.22279992699623108
Epoch #7: validation loss=0.10327749699354172

Epoch #8: train loss=0.2128724753856659
Epoch #8: validation loss=0.10261518508195877

Epoch #9: train loss=0.20037129521369934
Epoch #9: validation loss=0.10228367149829865

Epoch #10: train loss=0.19005003571510315
Epoch #10: validation loss=0.09972216188907623

Epoch #11: train loss=0.1844574213027954
Epoch #11: validation loss=0.09715581685304642

Epoch #12: train

In [16]:
print(train_repr.shape, test_repr.shape)

(7352, 64) (2947, 64)


In [17]:
train_repr.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V55,V56,V57,V58,V59,V60,V61,V62,V63,V64
0,0.067811,0.630309,0.678385,-0.224692,-0.494843,-0.082273,-0.337692,-0.141722,-0.223741,1.456698,...,-0.304692,-0.011693,-0.365004,-0.390378,-0.368446,0.667603,0.164786,-0.119798,0.028371,0.217439
1,0.072153,0.630751,0.686749,-0.223636,-0.507986,-0.094064,-0.350572,-0.141803,-0.219542,1.452028,...,-0.300298,-0.020464,-0.360855,-0.393285,-0.382104,0.660885,0.158050,-0.120539,0.032266,0.227818
2,0.072153,0.638612,0.686749,-0.223636,-0.509670,-0.094064,-0.350733,-0.141803,-0.219542,1.452028,...,-0.300298,-0.007431,-0.357887,-0.395162,-0.381343,0.669121,0.157673,-0.120539,0.032266,0.227818
3,0.062448,0.638658,0.674399,-0.224589,-0.515836,-0.098094,-0.362724,-0.149598,-0.215638,1.437033,...,-0.310291,-0.007431,-0.355633,-0.395879,-0.381343,0.670142,0.158143,-0.138508,0.017491,0.214453
4,0.064106,0.640889,0.685494,-0.226771,-0.515336,-0.089185,-0.353286,-0.138173,-0.211655,1.447111,...,-0.310015,-0.006760,-0.355429,-0.398926,-0.381573,0.671615,0.158277,-0.128860,0.022683,0.219661


In [18]:
test_repr.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V55,V56,V57,V58,V59,V60,V61,V62,V63,V64
0,0.111111,0.574629,0.770907,-0.245455,-0.387932,-0.004334,-0.301693,-0.052747,-0.156665,1.473217,...,-0.303291,-0.076165,-0.333392,-0.430074,-0.289225,0.759626,0.182444,-0.070789,0.011681,0.244707
1,0.109244,0.623049,0.738260,-0.245455,-0.438132,-0.072092,-0.335119,-0.116647,-0.212243,1.453629,...,-0.303291,-0.012675,-0.338566,-0.425654,-0.342676,0.718123,0.170675,-0.089175,0.011681,0.244707
2,0.089741,0.623049,0.697248,-0.248375,-0.460601,-0.099862,-0.367788,-0.153542,-0.221222,1.424201,...,-0.322693,-0.001682,-0.332708,-0.423858,-0.341373,0.729697,0.173032,-0.118748,-0.014104,0.203656
3,0.089741,0.629642,0.697065,-0.252234,-0.460601,-0.099238,-0.367788,-0.153542,-0.209767,1.424201,...,-0.329457,0.026987,-0.321140,-0.422830,-0.337990,0.745214,0.174257,-0.118748,-0.016303,0.194724
4,0.064320,0.629642,0.704274,-0.252234,-0.480357,-0.080069,-0.369631,-0.133919,-0.194449,1.420977,...,-0.346233,0.026987,-0.320160,-0.422830,-0.337990,0.745214,0.174257,-0.137089,-0.038669,0.175696


In [20]:
# Case 4. model = stoc
config = config4
data_repr = mdr.Encode(config, train_data, test_data)
model = data_repr.build_model()  # 모델 구축

if config["training"]:
    best_model = data_repr.train_model(model)  # 모델 학습
    data_repr.save_model(best_model, best_model_path=config["best_model_path"])  # 모델 저장

train_repr, test_repr = data_repr.encode_data(model, best_model_path=config["best_model_path"])  # representation 도출

Start training model

Epoch #1: loss=2.0070038635064575
Epoch #1: validation loss=0.11913683079183102

Epoch #2: loss=0.10930594363633324
Epoch #2: validation loss=0.10193572379648685

Epoch #3: loss=0.10185562030357473
Epoch #3: validation loss=0.09928877651691437

Epoch #4: loss=0.10030568084296058
Epoch #4: validation loss=0.09756442345678806

Epoch #5: loss=0.0994102889124085
Epoch #5: validation loss=0.09733622893691063

Epoch #6: loss=0.09906453902230543
Epoch #6: validation loss=0.09697526507079601

Epoch #7: loss=0.09919325087000341
Epoch #7: validation loss=0.09693634137511253

Epoch #8: loss=0.09855554647305433
Epoch #8: validation loss=0.0967323575168848

Epoch #9: loss=0.0968994230908506
Epoch #9: validation loss=0.09242820739746094

Epoch #10: loss=0.09095856635009542
Epoch #10: validation loss=0.08500596322119236

Epoch #11: loss=0.08550137894995072
Epoch #11: validation loss=0.08121322467923164

Epoch #12: loss=0.08093429751255933
Epoch #12: validation loss=0.07434214279

In [21]:
print(train_repr.shape, test_repr.shape)

(7352, 64) (2947, 64)


In [22]:
train_repr.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V55,V56,V57,V58,V59,V60,V61,V62,V63,V64
0,-0.398952,0.290622,0.324225,-0.394981,-0.437825,0.208608,-1.034431,-0.938719,-0.908528,1.510468,...,-0.377648,0.242635,0.436007,-0.819171,-0.615345,0.205519,-0.008637,-0.024010,-0.719357,0.465744
1,-0.398952,0.284166,0.324225,-0.394981,-0.438264,0.208608,-1.034431,-0.938719,-0.908528,1.499745,...,-0.376878,0.242635,0.431513,-0.818394,-0.614960,0.199832,-0.006871,-0.024010,-0.719357,0.465744
2,-0.387353,0.284166,0.340648,-0.377317,-0.436637,0.212478,-1.032836,-0.931129,-0.905558,1.496878,...,-0.376878,0.255813,0.431513,-0.817827,-0.614960,0.199832,-0.006871,-0.016742,-0.714316,0.474150
3,-0.387353,0.276282,0.340648,-0.377317,-0.433730,0.214189,-1.032836,-0.931129,-0.905558,1.475613,...,-0.377878,0.261039,0.430023,-0.815575,-0.615355,0.177819,-0.015549,-0.014781,-0.714316,0.474150
4,-0.389302,0.276282,0.332543,-0.380108,-0.433650,0.214189,-1.032949,-0.937065,-0.906430,1.484461,...,-0.380463,0.261039,0.430023,-0.815529,-0.615725,0.179552,-0.015549,-0.013870,-0.719365,0.470728


In [23]:
test_repr.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V55,V56,V57,V58,V59,V60,V61,V62,V63,V64
0,-0.421725,0.429874,0.205252,-0.420332,-0.391113,0.205659,-1.018567,-0.995579,-0.906346,1.724268,...,-0.401557,0.238907,0.554475,-0.764480,-0.611959,0.300524,0.028026,-0.011310,-0.761547,0.408547
1,-0.389708,0.415705,0.286972,-0.376126,-0.393209,0.218218,-1.018567,-0.968899,-0.901835,1.566057,...,-0.400865,0.280335,0.504924,-0.764480,-0.611959,0.300524,0.028026,0.005134,-0.740072,0.445111
2,-0.389708,0.343615,0.286972,-0.373838,-0.399181,0.218976,-1.020280,-0.968899,-0.901835,1.509274,...,-0.400865,0.281210,0.471867,-0.786482,-0.617629,0.221537,-0.003782,0.010128,-0.740072,0.445111
3,-0.372006,0.343615,0.300817,-0.349442,-0.399181,0.225255,-1.014195,-0.959147,-0.894597,1.509274,...,-0.402881,0.306219,0.471867,-0.785659,-0.615510,0.221537,-0.003782,0.016148,-0.737734,0.450091
4,-0.372006,0.317695,0.300817,-0.349442,-0.398042,0.225255,-1.013953,-0.959147,-0.894597,1.468945,...,-0.402881,0.306219,0.469023,-0.781360,-0.615510,0.173415,-0.018760,0.016148,-0.737734,0.450091
